In [3]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [4]:
sqlContext = pyspark.SQLContext(sc)
business = sqlContext.read.json('C:\\bigdata\data\yelp_dataset\\business.json')
review = sqlContext.read.json('C:\\bigdata\data\yelp_dataset\\review.json')
#checkin = sqlContext.read.json('C:\\bigdata\data\yelp_dataset\checkin.json')
#tip = sqlContext.read.json('C:\\bigdata\data\yelp_dataset\\tip.json')
#photo = sqlContext.read.json('C:\\bigdata\data\yelp_dataset\\photo.json')
#user = sqlContext.read.json('C:\\bigdata\data\yelp_dataset\\user.json')

In [147]:
help(review)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(builtins.object)
 |  DataFrame(jdf, sql_ctx)
 |  
 |  A distributed collection of data grouped into named columns.
 |  
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SparkSession`::
 |  
 |      people = spark.read.parquet("...")
 |  
 |  Once created, it can be manipulated using the various domain-specific-language
 |  (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 |  
 |  To select a column from the data frame, use the apply method::
 |  
 |      ageCol = people.age
 |  
 |  A more concrete example::
 |  
 |      # To create DataFrame using SparkSession
 |      people = spark.read.parquet("...")
 |      department = spark.read.parquet("...")
 |  
 |      people.filter(people.age > 30).join(department, people.deptId == department.id) \
 |        .groupBy(department.name, "gender").agg({"salary": "avg", "age

In [57]:
review.head(1)

[Row(business_id='ujmEBvifdJM6h6RLv4wQIg', cool=0, date='2013-05-07 04:34:36', funny=1, review_id='Q1sbwvVQXV2734tPgoKj4Q', stars=1.0, text='Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', useful=6, user_id='hG7b0MtEbXx5QzbzE6C_VA')]

In [5]:
business.printSchema()
# review.printSchema()
#checkin.printSchema()
#tip.printSchema()
#user.printSchema()
#photo.printSchema()

#review.head(10)

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [140]:
review.head(2)

[Row(business_id='ujmEBvifdJM6h6RLv4wQIg', cool=0, date='2013-05-07 04:34:36', funny=1, review_id='Q1sbwvVQXV2734tPgoKj4Q', stars=1.0, text='Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', useful=6, user_id='hG7b0MtEbXx5QzbzE6C_VA'),
 Row(business_id='NZnhc2sEQy3RmzKTZnqtwQ', cool=0, date='2017-01-14 21:30:33', funny=0, review_id='GJXCdrto3ASJOqKeVWPi6Q', stars=5.0, text="I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  \n\nTravis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure

In [4]:
 # Create a view to query the business
business.createOrReplaceTempView("business")
review.createOrReplaceTempView("review")

In [15]:
business_df=spark.sql("select * from business where business_id='1SWheh84yJXfytovILXOAQ'")

In [29]:
top_business=spark.sql("select business_id from business order by review_count desc limit 1").collect()

In [30]:
print(top_business)

[Row(business_id='4JNXUYY8wbaaDmk3BPzlWw')]


In [33]:
top_business

[Row(business_id='4JNXUYY8wbaaDmk3BPzlWw')]

In [37]:
spark.sql("select categories,count(business_id) as cnt from business group by 1 order by cnt desc limit 5").show()

+--------------------+----+
|          categories| cnt|
+--------------------+----+
|  Restaurants, Pizza|1042|
|Nail Salons, Beau...|1031|
|  Pizza, Restaurants| 993|
|Beauty & Spas, Na...| 947|
|  Food, Coffee & Tea| 888|
+--------------------+----+



In [39]:
spark.sql("select business_id from business where categories='Restaurants, Pizza' order by review_count desc limit 5").collect()

[Row(business_id='YQ--LJ7pvjiDSqNv0TuKTQ'),
 Row(business_id='UT6L3b7Zll_nvRidijiDSA'),
 Row(business_id='u_8cVZyxh0J468zEZUjNDQ'),
 Row(business_id='RRw9I8pHt5PzgYGT2QeODw'),
 Row(business_id='I0f12eU-xFn6Kd_4H57jrQ')]

In [9]:
review_subset=spark.sql("select r.* from review r inner join business b on r.business_id=b.business_id where categories='Restaurants, Pizza' and r.stars<2 and text is not null")

In [143]:
import json
import csv
import re
import string
import pandas as pd

reviews_pd=review_subset.toPandas()

#reviews = open("C:\\bigdata\data\yelp_dataset\\review.json",encoding="utf8")

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

c=0
business_review={}

for index, row in reviews_pd.iterrows():    
    if c==-9999:
        break
    else:
        if row['business_id'] in business_review:
            all_review_text=business_review[row['business_id']]
            current_review=row['text']
            current_review=clean_text(current_review)
            all_review_text=all_review_text+' '+current_review            
            business_review[row['business_id']]=all_review_text
        else:
            current_review=row['text']
            current_review=clean_text_round1(current_review)
            business_review[row['business_id']]=current_review
    c=c+1

#for line in reviews:
#    json_parsed = json.loads(line)
#    if c==9999:
#        break
#    else:
#        if json_parsed['business_id'] in business_review:
#            all_review_text=business_review[json_parsed['business_id']]
#            current_review=json_parsed['text']
#            current_review=clean_text(current_review)
#            all_review_text=all_review_text+' '+current_review            
#            business_review[json_parsed['business_id']]=all_review_text
#        else:
#            current_review=json_parsed['text']
#            current_review=clean_text_round1(current_review)
#            business_review[json_parsed['business_id']]=current_review
#    c=c+1

In [94]:
#print(business_review.keys())
#print(business_review.values())

print(business_review['ujmEBvifdJM6h6RLv4wQIg'])
#print(business_review['UT6L3b7Zll_nvRidijiDSA'])

#next(iter(business_review.keys()))


total bill for this horrible service over  these crooks actually had the nerve to charge us  for  pills i checked online the pills can be had for  cents each avoid hospital ers at all costs my family has used this er four times in the past  years once you get past the waiting room this is a decent er the paid staff docs nurses med assistants etc all seem to have their act together they have volunteers at the front desk but i dont know what their purpose is because the one i talked to just sat their playing with his phone and wasnt able to provide any assistance to me the checkin can be confusing because its all done at a kiosk and during an emergency the last thought on your mind is to walk into an er only to have your first contact be with a kiosk instead of a human lastly i wish they had an onsite pharmacy that can fill prescriptions in the middle of the night its frustrating to spend hours in the er and then have to go find an open pharmacy that accepts your insurance overall though

In [10]:
reviews_pd=review_subset.toPandas()

In [113]:
reviews_pd['business_id']

0        E4t43bnsfeOY2bv_8IPmSw
1        k2b3niokS_tosjah_rzCPw
2        k2b3niokS_tosjah_rzCPw
3        oD8kSlXINk5u7d6HQh2SxQ
4        M5Nvx_jkTf56Ne7tuXgCLQ
                  ...          
27132    EVjwTJ3boXTve0hTTFxPJw
27133    ckZg3KVwR9FrAR1KtTavEQ
27134    JZ4ZRdRSllhuAt94tFCH8w
27135    VkPa6mYsHsiPPU6BQ3l4fQ
27136    VkPa6mYsHsiPPU6BQ3l4fQ
Name: business_id, Length: 27137, dtype: object

In [12]:
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

#cv = CountVectorizer(stop_words='english')
#data_cv = cv.fit_transform(reviews_pd.text)
#data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
#data_dtm.index = reviews_pd.index
#data_dtm

In [17]:
# Let's make some word clouds!
# Terminal / Anaconda Prompt: conda install -c conda-forge wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

stop_words = text.ENGLISH_STOP_WORDS.union(['pizza','slice','told','order','ordered','food','just','I''m','going','like','really','asked','place','time','got','called','came'])

wc = WordCloud(stopwords=stop_words, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

# print(reviews_pd['text'][0])
wc.generate(' '.join(reviews_pd['text']))

plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
# plt.show()
plt.savefig("C:\\bigdata\data\yelp_dataset\plots\mygraph.png")

In [152]:
import os

os.cpu_count()

8

In [153]:
import multiprocessing as ms
ms.cpu_count()

8

In [6]:
import pandas_profiling as pp

In [8]:
pp.ProfileReport(reviews_pd)

C:\Users\rohan\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,9
Number of observations,7230
Total Missing (%),0.0%
Total size in memory,508.5 KiB
Average record size in memory,72.0 B
Numeric,3
Categorical,3
Boolean,1
Date,0
Text (Unique),2
Rejected,0
